In [ ]:
import re
import math
import os
global stopWl

def acentos(s):
    ac = ["á","é","í","ó","ú","ñ"]
    s = s.replace(ac[0],"a")
    s = s.replace(ac[1],"e")
    s = s.replace(ac[2],"i")
    s = s.replace(ac[3],"o")
    s = s.replace(ac[4],"u")
    s = s.replace(ac[5],"n")
    return s

def minusculas(s):
    return s.lower()

def diccionario(lista,num):
    global dicc
    for i in lista:
        if i in dicc:
            dicc[i].append(num)
        else:
            dicc[i] = []
            dicc[i].append(num)
            
def plurarASingular(palabra):
    if len(palabra)> 3 and palabra[-2] == 'e':
        if palabra[-3] in ['i','u','b','c','g','m','p','t']:
            palabra = palabra[0:-2]
    if palabra[-1] == 's'and palabra[-2] in ['a','e','o','u']:
            palabra = palabra[0:-1]
    return palabra

    

def procesar(i):
    global stopWl
    l = acentos(i)
    l= re.sub('\W+',' ',l)
    l = minusculas(l)
    lista = l.split()
    aux = []
    for i in lista:
        if len(i) > 2 and i not in stopWl:
            if i[-1]=="s":
                i =plurarASingular(i)
            aux.append(i)
    return aux

def contruirifidf(tf,idf):
    llaves = tf.keys()
    dicc = {}
    for i in llaves:
        dicc[i] = tf[i]*idf[i]
        #print(i," : ", dicc[i])
    return dicc
        
def leerStopW(archi):
    stopWl =[]
    for i in archi.readlines():
        i=i[0:-1]
        stopWl.append(i)
    return stopWl    
    
    
def main():
    global stopWl, dicc
    dicc = {}
    dicc_tf = {}
    dicc_idf = {}
    dicc_tf_idf= {}
    stopWl=[]
    documentos = []
    documentosOriginales = []
    
    stopW = open("StopWords.txt", "r+")
    stopWl = leerStopW(stopW)
    stopW.close()

    #archi = open("test.txt",encoding='utf8')
    
    path = "C:\\MGVD\\FC\\"    
    files = os.listdir(path)
    
    aux =0
    for i in files:
        file_path = "C:\\MGVD\\FC\\" + i
        file = open(file_path , encoding="utf-8")
        h = file.read()
        documentosOriginales.append(h)
        l = procesar(h)
        documentos.append(l)
        diccionario(l,aux)
        # contar el tf
        size = len(l)
        for j in l:
            eux = l.count(j)
            dicc_tf[j+" "+str(aux)] = float(eux) / float(size)
        aux += 1
        
    aux +=1
    iux = 0
    for j in documentos:
        for i in j:
            dicc_idf[i+" "+str(iux)] = math.log10(float(aux)/float(len(dicc[i])))
        iux+=1

    dicc_tf_idf = contruirifidf(dicc_tf,dicc_idf)

    
    while (True):
        consulta = str(input("Escribe tu consulta: "))
        consulta = procesar(consulta)
        size = len(consulta)
        dicc_cons_tf = {}
        dicc_cons_idf = {}
        

        for i in consulta:
            dicc_cons_tf[i]=float(consulta.count(i)) / float(size)
            try :
                dicc_cons_idf[i] = math.log10(float(aux)/float(len(dicc[i])))
            except :
                dicc_cons_idf[i]= 0

        dicc_constfidf= contruirifidf(dicc_cons_tf,dicc_cons_idf)
        
        q = dicc_constfidf.keys()
        eux = 0
        mejor = 0
        resultados = []
        for i in range(aux):
            for j in q:
                myll=j+" "+str(i)
                try :
                    eux += dicc_constfidf[j] * dicc_tf_idf[myll]
                except:
                    eux = eux
            if eux > 0 :#and eux>mejor:
                l = [eux, documentosOriginales[i],i]
                if resultados != []:
                    if resultados[0][0]<l[0]:
                        resultados.insert(0,l)
                    elif resultados[-1][0]>l[0]:
                            resultados.append(l)
                    else:
                        a = 0
                        for i in resultados:
                            if i[0]<l[0]:
                                resultados.insert(a,l)
                                break
                            a += 1
                else:
                    resultados.append(l)
##                mejor = eux
##                print(eux)
##                print(documentos[i])
##                try : 
##                    print(documentosOriginales[i])
##                except:
##                    print ("error al escribir el documento")
            eux = 0
        for i in resultados:
            print(i)
        
main()

Escribe tu consulta: ensenada
[0.00981842461851716, 'Los restos de la impronta española en Filipinas\nCompartir en Facebook\n18 OCT 2011\nAcabo de regresar de un viaje a las Filipinas (el vuelo tomó 27 horas desde Estados Unidos), donde fui a investigar el sistema de pensiones. Solo estuve en "Metro Manila" que comprende 15 ciudades con 16 millones de habitantes, de una población total de 90 millones. No pude visitar las bellezas naturales en el norte y en el sur.\nEn 1564 comenzó la colonización española que duró más de 300 años. Para que los frailes pudiesen identificar a los nativos, les dieron a escoger de una lista de nombres castellanos, que aún hoy perduran en los filipinos y en las calles; pero el castellano no logró sustituir a los dialectos indígenas. Manila floreció con los galeones que intercambiaban especies y mercancías chinas por el oro y plata mexicanos desde el puerto de Acapulco, pero el lucrativo intercambio desapareció a fines del siglo XVIII.\nHoy casi nadie habla 

Escribe tu consulta: Los filipinos estuvieron expuestos a las artes
[0.006916423958283859, '► Philippines in art \u200e (1 C,  7 F)\nB\n► Books about the Philippines \u200e (19 C,  4 F)\nC\n► Culture of the Philippines \u200e (63 C,  5 F)\nD\n► Documents of the Philippines \u200e (5 C,  5 F)\nE\n► Environment of the Philippines \u200e (1 C,  1 F)\nG\n► Geography of the Philippines \u200e (22 C,  2 F)\nH\n► History of the Philippines \u200e (53 C,  114 F)\nI\n', 56]
[0.005993154864181562, 'Universidades\nInformación\nUna de las características más notables de la sociedad filipina es la                  tradición de una fuerte fidelidad familiar. Esto se refleja en la ausencia                  de instituciones geriátricas, dedicadas al cuidado de personas mayores.\nCultura\nLas artes como la música formaban parte de la vida pre-hispánica            en Filipinas. Los instrumentos musicales como la flauta y la guitarra de            bambú y el gong, eran utilizados en épocas que datan del 

Escribe tu consulta: estuvieron expuestos a las artes Los filipinos 
[0.006916423958283859, '► Philippines in art \u200e (1 C,  7 F)\nB\n► Books about the Philippines \u200e (19 C,  4 F)\nC\n► Culture of the Philippines \u200e (63 C,  5 F)\nD\n► Documents of the Philippines \u200e (5 C,  5 F)\nE\n► Environment of the Philippines \u200e (1 C,  1 F)\nG\n► Geography of the Philippines \u200e (22 C,  2 F)\nH\n► History of the Philippines \u200e (53 C,  114 F)\nI\n', 56]
[0.005993154864181562, 'Universidades\nInformación\nUna de las características más notables de la sociedad filipina es la                  tradición de una fuerte fidelidad familiar. Esto se refleja en la ausencia                  de instituciones geriátricas, dedicadas al cuidado de personas mayores.\nCultura\nLas artes como la música formaban parte de la vida pre-hispánica            en Filipinas. Los instrumentos musicales como la flauta y la guitarra de            bambú y el gong, eran utilizados en épocas que datan del